<a href="https://colab.research.google.com/github/harshilj0310/Data-Science-Notebooks/blob/main/assignment_3(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Define constants
SEED = 0
NUM_FEATURES = 6  # Number of features used
STOP_WORDS = stopwords.words("english")
PUNCTUATION_MARKS = list(punctuation)

In [ ]:
def vectorize_word(token, scaled_index):
    # token : str : a word

    vector = np.zeros(NUM_FEATURES).astype(np.float32)

    # If first character is uppercase
    is_title = 1 if token[0].isupper() else 0

    # All characters in uppercase
    is_allcaps = 1 if token.isupper() else 0

    # Is stopword
    is_stopword = 1 if token.lower() in STOP_WORDS else 0

    # Is punctuation
    is_punctuation = 1 if token in PUNCTUATION_MARKS else 0

    # Build feature vector
    vector[0] = is_title
    vector[1] = is_allcaps
    vector[2] = len(token)
    vector[3] = is_stopword
    vector[4] = is_punctuation
    vector[5] = scaled_index

    return vector


In [ ]:
def prepare_data(dataset):
    word_list = []  # Stores the words as strings
    feature_vectors = []  # Feature array, one vector per word in the dataset
    labels_list = []  # Labels (0/1)

    for entry in tqdm(dataset):
        words = entry["tokens"]
        ner_tags = entry["ner_tags"]

        num_tokens = len(words)
        for index in range(num_tokens):
            feature_vector = vectorize_word(token=words[index], scaled_index=(index / num_tokens))

            label = 1 if ner_tags[index] > 0 else 0

            feature_vectors.append(feature_vector)
            labels_list.append(label)

        word_list += words

    word_list = np.asarray(word_list, dtype="object")
    feature_vectors = np.asarray(feature_vectors, dtype=np.float32)
    labels_list = np.asarray(labels_list, dtype=np.float32)

    return word_list, feature_vectors, labels_list

In [ ]:
def infer_labels(trained_model, feature_scaler, sentence):
    # sentence: input string

    tokens = word_tokenize(sentence)
    feature_vectors = []

    num_tokens = len(tokens)
    for index in range(num_tokens):
        feature_vector = vectorize_word(token=tokens[index], scaled_index=(index / num_tokens))
        feature_vectors.append(feature_vector)

    feature_vectors = np.asarray(feature_vectors, dtype=np.float32)
    scaled_features = feature_scaler.transform(feature_vectors)
    predictions = trained_model.predict(scaled_features)

    return predictions, tokens, feature_vectors

In [ ]:
# Load the dataset
dataset = load_dataset("conll2003", trust_remote_code=True)  # Type: datasets.dataset_dict.DatasetDict
train_data = dataset["train"]  # 14,041 rows
validation_data = dataset["validation"]  # 3250 rows
test_data = dataset["test"]  # 3453 rows

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
# Columns: 'id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'
train_words, X_train, y_train = prepare_data(train_data)
val_words, X_val, y_val = prepare_data(validation_data)
test_words, X_test, y_test = prepare_data(test_data)

100%|██████████| 3453/3453 [00:01<00:00, 2793.37it/s]


In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(203621, 6)
(51362, 6)
(46435, 6)


In [ ]:
# Print some examples of named entities in y_val
named_entities = val_words[y_val == 1]
for ne in np.random.choice(named_entities, size=15):
    print(ne)

Staunton
Pirelli
Kushko
Khan
Steve
Detroit
Browne
France
England
Nasdaq
Ramaunaskaite
Department
Bloomfield
New
SEOUL


In [ ]:
# Standardize the features so that all features contribute equally to the distance metric computation of the SVM
scaler = StandardScaler()

In [ ]:
# Fit only on the training data (i.e., compute mean and standard deviation)
scaler = scaler.fit(X_train)

In [ ]:
# Use the train data fit values to scale validation and test data
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Train the SVM model
svm_model = SVC(C=1.0, kernel="linear", class_weight="balanced", random_state=SEED, verbose=True)
svm_model.fit(X_train, y_train)

[LibSVM]

In [ ]:
import joblib

# Save the SVM model
joblib.dump(svm_model, 'svm_model.joblib')

In [ ]:
# Load the SVM model
svm_model = joblib.load('svm_model.joblib')

In [ ]:
# Predict on the validation set
y_pred_val = svm_model.predict(X_val)

print(classification_report(y_true=y_val, y_pred=y_pred_val))

In [ ]:
saved_y_pred = np.array(y_pred_val, dtype=object)
np.save('y_pred_saved.npy', saved_y_pred)

In [ ]:
y_pred1 = np.load('y_pred_saved.npy', allow_pickle=True)
print(y_pred1)

In [ ]:
# Test with example sentences
example_sentences = [
    "Delhi is the capital of India.",
    "US Vice President Kamala Harris, PM Modi talk up Indo-US ties at 1st in-person meeting.",
    "Covid-19 India Live News: National Task Force drops Ivermectin, HCQ drugs from Covid-19 treatment protocol; India logs 31,382 new cases.",
    "US Rules Out Adding India Or Japan To Security Alliance With Australia And UK"
]

for sentence in example_sentences:
    predictions, tokens, features = infer_labels(svm_model, scaler, sentence)
    annotated_sentence = []
    for word, prediction in zip(tokens, predictions):
        annotated_sentence.append(f"{word}_{int(prediction)}")
    print(" ".join(annotated_sentence))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Generate predictions on the validation set if not already done
#y_pred_val = svm_model.predict(X_val)

# Compute the confusion matrix
cm = confusion_matrix(y_val, y_pred_val)

# Plot the confusion matrix with seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Non-Entity", "Entity"], yticklabels=["Non-Entity", "Entity"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for SVM Named Entity Recognition")
plt.show()


In [ ]:
import gradio as gr
import joblib

# Load the saved model and scaler
svm_model = joblib.load('svm_model.joblib')
#scaler = joblib.load('scaler.joblib')  # Assuming you've saved the scaler similarly

def annotate_entities(sentence):
    # Use the previously defined infer_labels function
    predictions, tokens, features = infer_labels(svm_model, scaler, sentence)

    # Annotate each token based on predictions
    annotated_sentence = []
    for word, prediction in zip(tokens, predictions):
        annotated_sentence.append(f"{word}_{int(prediction)}")

    return " ".join(annotated_sentence)

# Set up the Gradio interface
iface = gr.Interface(
    fn=annotate_entities,          # Function to use for prediction
    inputs="text",                  # Input type
    outputs="text",                 # Output type
    title="Named Entity Identifier",
    description="Enter a sentence to identify named entities using an SVM model."
)

# Launch the Gradio app
iface.launch()
